In [18]:
# !pip install noisereduce -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import torch
import librosa
import numpy as np
import glob
import os
from tqdm import tqdm
import pandas as pd

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio
from secs import SpeechObjectiveEvaluation

from IPython.display import clear_output


In [2]:
sr = 16000
models = ["resemblyzer", "wavlm", "wespeaker", "wespeaker_lm", "wespeaker_nolm"]
soe = SpeechObjectiveEvaluation(sr, models)

Some weights of the model checkpoint at /mntcephfs/data/audiow/shoinoue/Model/hf_hub/wavlm/models--microsoft--wavlm-base-plus-sv/snapshots/feb593a6c23c1cc3d9510425c29b0a14d2b07b1e/ were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at /mntcephfs/data/audiow/shoinoue/Model/hf_hub/wavlm/models--microsoft--wavlm-base-plus-sv/snapshots/feb593a6c23c1cc3d9510425c29b0a1

Loaded the voice encoder model on cuda in 2.62 seconds.


In [9]:
##### gt: CMU-ARCTIC, english: Elevenlabs English, hindi: Elevellabs transliterated Hindi
# dir1 = "/mntcephfs/lab_data/shoinoue/Dataset/CMU-ARCTIC/SLT/wav/"
# dir2 = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/English/wav/"
# dir3 = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"

##### gt: L2-ARCTIC, english: CosyVoice cross lingual version (xvector), hindi: CosyVoice zeroshot version (speechprompt+xvector)
dir1 = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/wav/"
dir2 = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/cosyvoice/wav/"
dir3 = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/cosyvoice/wavzs/"

sample_num = 2
simdir = {
    "gt-english": [dir1, dir2],
    "gt-hindi": [dir1, dir3],
    "english-hindi": [dir2, dir3],
}
scores = {}
for la in simdir:
    base1, base2 = simdir[la]
    b = [os.path.basename(a) for a in glob.glob(base1+"*.wav")]
    commons = [a for a in [os.path.basename(a) for a in glob.glob(base2+"*.wav")] if a in b]
    commons.sort()
    commons = commons[:sample_num]
    
    score = {a: [] for a in models}
    for i, bn in tqdm(enumerate(commons[1:])):
        file1 = base1 + bn
        file2 = base2 + bn
        play_audio(file1, sr)
        play_audio(file2, sr)
        sim = soe.get_speaker_similarity(file1, file2)
        for key in sim:
            score[key] += [sim[key]]
    scores[la] = score
    
sims = scores.copy()
for la in sims:
    for key in sims[la]:
        sims[la][key] = np.array(sims[la][key]).mean()
pd.DataFrame(sims)

0it [00:00, ?it/s]

1it [00:04,  4.99s/it]
0it [00:00, ?it/s]

1it [00:01,  1.79s/it]
0it [00:00, ?it/s]

1it [00:01,  1.03s/it]


,gt-english,gt-hindi,english-hindi
resemblyzer,0.832311,0.846639,0.803468
wavlm,0.682689,0.956976,0.787494
wespeaker,0.492170,0.601550,0.525402
wespeaker_lm,0.574560,0.622798,0.611340
wespeaker_nolm,0.574523,0.628996,0.624723
